In [3]:
import numpy as np
import pandas as pd

In [4]:
!pip install pyodbc pandas


In [5]:
!pip install sqlalchemy pyodbc pandas


In [6]:
from sqlalchemy import create_engine
import pandas as pd

# Define the connection string
connection_string = (
    "mssql+pyodbc://DESKTOP-45PMML3/AdventureWorks2022"
    "?driver=SQL+Server"
)

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

try:
    # Define SQL query
    query = "SELECT TOP 10 * FROM [HumanResources].[Department]"

    # Execute query and Load into DataFrame
    df = pd.read_sql(query, engine)

    # Show the results
    print(df.head())

except Exception as e:
    print(f"Error: {e}")


   DepartmentID         Name                 GroupName ModifiedDate
0             1  Engineering  Research and Development   2008-04-30
1             2  Tool Design  Research and Development   2008-04-30
2             3        Sales       Sales and Marketing   2008-04-30
3             4    Marketing       Sales and Marketing   2008-04-30
4             5   Purchasing      Inventory Management   2008-04-30


In [7]:

# 1. Select EmployeeID, FirstName, and LastName columns from the Employee DataFrame
query1 = """
SELECT e.BusinessEntityID as EmployeeID, p.FirstName, p.LastName
FROM HumanResources.Employee e
JOIN Person.Person p
ON e.BusinessEntityID = p.BusinessEntityID
"""
employee_df = pd.read_sql(query1, engine)
print(employee_df)

     EmployeeID FirstName         LastName
0            89   Patrick            Wedge
1           120     Kitti  Lertpiriyasuwat
2             9      Gigi          Matthew
3           205      Lori             Kane
4            71   Michael              Ray
..          ...       ...              ...
285          82      Jack          Creasey
286         157     Linda          Randall
287          95       Jim        Scardelis
288         215      Mark       Harrington
289         112      John            Evans

[290 rows x 3 columns]


In [8]:
# 2. Sort the Employee DataFrame by LastName in ascending order
sorted_employee_df = employee_df.sort_values(by='LastName')
print(sorted_employee_df)

     EmployeeID FirstName     LastName
59          285      Syed        Abbas
251          38       Kim  Abercrombie
142         211     Hazem     Abolrous
239         121     Pilar     Ackerman
240          67       Jay        Adams
..          ...       ...          ...
248          99      Nuan           Yu
212         129      Gary       Yukish
39          173    Eugene  Zabokritski
56           91  Kimberly    Zimmerman
234          64   Michael     Zwilling

[290 rows x 3 columns]


In [9]:
# 3. Retrieve distinct JobTitle values from the Employee DataFrame
query2 = """
SELECT DISTINCT JobTitle
FROM HumanResources.Employee
"""
job_titles_df = pd.read_sql(query2, engine)
print(job_titles_df)

                          JobTitle
0                       Accountant
1                 Accounts Manager
2      Accounts Payable Specialist
3   Accounts Receivable Specialist
4           Application Specialist
..                             ...
62                         Stocker
63                   Tool Designer
64   Vice President of Engineering
65    Vice President of Production
66         Vice President of Sales

[67 rows x 1 columns]


In [20]:
# 4. Filter employees with JobTitle containing 'Manager'
query3 = """
SELECT e.BusinessEntityID as EmployeeID, p.FirstName, p.LastName, e.JobTitle
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
WHERE e.JobTitle LIKE '%Manager%'
"""
manager_df = pd.read_sql(query3, engine)
print(manager_df)

    EmployeeID  FirstName           LastName                          JobTitle
0            3    Roberto         Tamburello               Engineering Manager
1            7      Dylan             Miller  Research and Development Manager
2           10    Michael             Raheem  Research and Development Manager
3           16      David            Bradley                 Marketing Manager
4           26      Peter              Krebs        Production Control Manager
5          211      Hazem           Abolrous         Quality Assurance Manager
6          217     Zainal             Arifin          Document Control Manager
7          227       Gary             Altman                Facilities Manager
8          235      Paula  Barreto de Mattos           Human Resources Manager
9          241      David                Liu                  Accounts Manager
10         249      Wendy               Kahn                   Finance Manager
11         250     Sheela               Word        

In [21]:
# 5. Count the total number of employees in the Employee DataFrame
query4 = """
SELECT COUNT(*) as TotalEmployees
FROM HumanResources.Employee
"""
total_employees_df = pd.read_sql(query4, engine)
print(total_employees_df)

   TotalEmployees
0             290


In [36]:
# 6. Find employees who are currently on leave.				
query6 = """
SELECT BusinessEntityID
FROM HumanResources.Employee
WHERE CurrentFlag = 0;
"""
onleave_df = pd.read_sql(query6, engine)
print(onleave_df)

Empty DataFrame
Columns: [BusinessEntityID]
Index: []


In [22]:
# 7. Check for any missing hire dates in the Employee DataFrame
query7 = """
SELECT e.BusinessEntityID as EmployeeID, p.FirstName, p.LastName, eph.RateChangeDate as HireDate
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.EmployeePayHistory eph ON e.BusinessEntityID = eph.BusinessEntityID
WHERE eph.RateChangeDate IS NULL
"""
missing_hire_dates_df = pd.read_sql(query7, engine)
print(missing_hire_dates_df)

Empty DataFrame
Columns: [EmployeeID, FirstName, LastName, HireDate]
Index: []


In [15]:
from datetime import datetime, timedelta

# 8. Select employees who are eligible for retirement
query8 = """
SELECT e.BusinessEntityID, p.FirstName, p.LastName, 
       DATEDIFF(YEAR, e.BirthDate, GETDATE()) AS Age
FROM HumanResources.Employee AS e
JOIN Person.Person AS p ON e.BusinessEntityID = p.BusinessEntityID
WHERE DATEDIFF(YEAR, e.BirthDate, GETDATE()) >= 60;


"""
retirement_eligible_df = pd.read_sql(query8, engine)
print(retirement_eligible_df)

    BusinessEntityID FirstName        LastName  Age
0                287       Amy         Alberts   67
1                282      José         Saraiva   61
2                277   Jillian          Carson   62
3                274   Stephen           Jiang   73
4                262     David          Barber   60
5                256     Frank          Pellow   72
6                231        Jo           Berry   70
7                206    Stuart          Munson   62
8                140  Prasanna  Samarawickrama   71
9                 84     Frank        Martinez   72
10                63    Maciej           Dusza   69
11                62      John        Campbell   68
12                55    Taylor         Maxwell   68
13                53     Diane           Glimp   68
14                52     Doris         Hartwig   68
15                51   Jeffrey            Ford   68
16                50    Sidney            Higa   68
17                49     Barry         Johnson   68
18          

In [28]:
# 9. Calculate the total number of employees in each department
query9 = """
SELECT edh.DepartmentID, d.Name as DepartmentName, COUNT(e.BusinessEntityID) as TotalEmployees
FROM HumanResources.Employee e
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
GROUP BY edh.DepartmentID, d.Name
"""
employees_per_department_df = pd.read_sql(query9, engine)
print(employees_per_department_df)

    DepartmentID              DepartmentName  TotalEmployees
0              1                 Engineering               7
1              2                 Tool Design               4
2              3                       Sales              18
3              4                   Marketing              10
4              5                  Purchasing              13
5              6    Research and Development               4
6              7                  Production             180
7              8          Production Control               6
8              9             Human Resources               6
9             10                     Finance              11
10            11        Information Services              10
11            12            Document Control               5
12            13           Quality Assurance               7
13            14  Facilities and Maintenance               7
14            15      Shipping and Receiving               6
15            16        

In [37]:
# 10. Merge the Employee and Department DataFrames on DepartmentID
query10 = """
SELECT e.BusinessEntityID as EmployeeID, p.FirstName, p.LastName, d.Name as DepartmentName
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
"""
employee_department_df = pd.read_sql(query10, engine)
print(employee_department_df)

     EmployeeID FirstName  LastName          DepartmentName
0           219      Sean      Chai        Document Control
1           220     Karen     Berge        Document Control
2           221     Chris    Norred        Document Control
3             5      Gail  Erickson             Engineering
4             6    Jossef  Goldberg             Engineering
..          ...       ...       ...                     ...
291         126     Jimmy  Bischoff  Shipping and Receiving
292           4       Rob   Walters             Tool Design
293          11    Ovidiu   Cracium             Tool Design
294          12   Thierry    D'Hers             Tool Design
295          13    Janice    Galvin             Tool Design

[296 rows x 4 columns]


In [40]:
from datetime import datetime, timedelta
import pandas as pd

# 11. Create an intermediate DataFrame to calculate the number of employees hired in the last 12 months
twelve_months_ago = datetime.now() - timedelta(days=365)

query11 = f"""
SELECT COUNT(e.BusinessEntityID) as NumberOfPeople
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
WHERE edh.StartDate >= '{twelve_months_ago.strftime('%Y-%m-%d')}' 
"""

# Execute the query and fetch the result
recently_hired_df = pd.read_sql(query11, engine)
print(recently_hired_df)
# max is 2012

   NumberOfPeople
0               0


In [41]:
# 12. Filter departments with more than 50 employees
query12 = """
SELECT edh.DepartmentID, d.Name as DepartmentName, COUNT(e.BusinessEntityID) as TotalEmployees
FROM HumanResources.Employee e
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
GROUP BY edh.DepartmentID, d.Name
HAVING COUNT(e.BusinessEntityID) > 50
"""
large_departments_df = pd.read_sql(query12, engine)
print(large_departments_df)

   DepartmentID DepartmentName  TotalEmployees
0             7     Production             180


In [16]:
# 13. Identify the average tenure of employees in each department
query13 = """
SELECT edh.DepartmentID, d.Name as DepartmentName, 
AVG(DATEDIFF(YEAR, e.HireDate, GETDATE())) as AvgTenureYears
FROM HumanResources.Employee e
JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
GROUP BY edh.DepartmentID, d.Name
"""
average_tenure_df = pd.read_sql(query13, engine)
print(average_tenure_df)

    DepartmentID              DepartmentName  AvgTenureYears
0              1                 Engineering              15
1              2                 Tool Design              15
2              3                       Sales              12
3              4                   Marketing              14
4              5                  Purchasing              14
5              6    Research and Development              15
6              7                  Production              15
7              8          Production Control              15
8              9             Human Resources              15
9             10                     Finance              15
10            11        Information Services              15
11            12            Document Control              15
12            13           Quality Assurance              14
13            14  Facilities and Maintenance              14
14            15      Shipping and Receiving              15
15            16        

In [48]:
# 14. Calculate the turnover rate for the company
query14 = """
SELECT 
    (SELECT COUNT(*) 
     FROM HumanResources.EmployeeDepartmentHistory 
     WHERE EndDate <= GETDATE()) as EmployeesLeft,
    (SELECT COUNT(*) 
     FROM HumanResources.Employee) as TotalEmployees
"""
turnover_df = pd.read_sql(query14, engine)
turnover_df['TurnoverRate'] = (turnover_df['TotalEmployees']/ turnover_df['EmployeesLeft']) * 100
print(turnover_df)


   EmployeesLeft  TotalEmployees  TurnoverRate
0              6             290   4833.333333


In [38]:
# 15. Determine the number of employees in each age group
query15 = """
SELECT 
CASE 
    WHEN DATEDIFF(YEAR, e.BirthDate, GETDATE()) < 30 THEN 'Under 30'
    WHEN DATEDIFF(YEAR, e.BirthDate, GETDATE()) BETWEEN 30 AND 50 THEN '30-50'
    ELSE 'Over 50'
END as AgeGroup, COUNT(*) as TotalEmployees
FROM HumanResources.Employee e
GROUP BY 
CASE 
    WHEN DATEDIFF(YEAR, e.BirthDate, GETDATE()) < 30 THEN 'Under 30'
    WHEN DATEDIFF(YEAR, e.BirthDate, GETDATE()) BETWEEN 30 AND 50 THEN '30-50'
    ELSE 'Over 50'
END
"""
age_group_df = pd.read_sql(query15, engine)
print(age_group_df)


  AgeGroup  TotalEmployees
0    30-50             215
1  Over 50              75


In [59]:
#16. Filter employees who have not completed mandatory training.				
query16="""
SELECT 
    p.FirstName,
    p.LastName,
    e.BusinessEntityID,
    eph.PayFrequency
FROM 
    Person.Person AS p
JOIN 
    HumanResources.Employee AS e ON p.BusinessEntityID = e.BusinessEntityID
JOIN 
    HumanResources.EmployeePayHistory AS eph ON e.BusinessEntityID = eph.BusinessEntityID
WHERE 
    eph.PayFrequency = 2;
"""

employee_temp_df = pd.read_sql(query16, engine)
print(employee_temp_df)

    FirstName    LastName  BusinessEntityID  PayFrequency
0        Jean     Trenary               263             2
1      Reuben        D'sa                78             2
2     Deborah         Poe               242             2
3    Matthias      Berndt               125             2
4     Garrett      Vargas               278             2
..        ...         ...               ...           ...
131       Rob     Walters                 4             2
132     Terri       Duffy                 2             2
133   Roberto  Tamburello                 3             2
134    Gordon         Hee               255             2
135      Mark  Harrington               215             2

[136 rows x 4 columns]


In [17]:
# 17. Rank employees by hire date
import pandas as pd
query17 = """
SELECT e.BusinessEntityID as EmployeeID, e.HireDate as HireDate
FROM HumanResources.Employee e
"""
employee_hire_df = pd.read_sql(query17, engine)

# Rank employees by hire date using pandas
employee_hire_df['Rank'] = employee_hire_df['HireDate'].rank(ascending=True)
print(employee_hire_df.sort_values('Rank'))

     EmployeeID    HireDate   Rank
27           28  2006-06-30    1.0
16           17  2007-01-26    2.0
2             3  2007-11-11    3.0
3             4  2007-12-05    4.0
11           12  2007-12-11    5.0
..          ...         ...    ...
289         290  2012-05-30  285.5
283         284  2012-09-30  287.0
284         285  2013-03-14  288.0
285         286  2013-05-30  289.5
287         288  2013-05-30  289.5

[290 rows x 3 columns]


In [18]:
# 18. Top 5 longest-serving employees
longest_serving_df = employee_hire_df.sort_values('HireDate').head(5)
print(longest_serving_df)


    EmployeeID    HireDate  Rank
27          28  2006-06-30   1.0
16          17  2007-01-26   2.0
2            3  2007-11-11   3.0
3            4  2007-12-05   4.0
11          12  2007-12-11   5.0


In [28]:
# 19. Function to retrieve employee details based on DepartmentID and HireDate
def get_employee_by_dept_and_hiredate(department_id, hire_date):
    query = f"""
    SELECT e.BusinessEntityID as EmployeeID, p.FirstName, p.LastName, d.Name as DepartmentName, e.HireDate
    FROM HumanResources.Employee e
    JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
    JOIN HumanResources.EmployeeDepartmentHistory edh ON e.BusinessEntityID = edh.BusinessEntityID
    JOIN HumanResources.Department d ON edh.DepartmentID = d.DepartmentID
    WHERE edh.DepartmentID = {department_id} AND e.HireDate = '{hire_date}'
    """
    return pd.read_sql(query, engine)

# Example usage
dept_employee_df = get_employee_by_dept_and_hiredate(16, '2009-01-14')
print(dept_employee_df)


   EmployeeID FirstName LastName DepartmentName    HireDate
0           1       Ken  Sánchez      Executive  2009-01-14


In [32]:
# 20. Days since last promotion (assuming last promotion is the most recent RateChangeDate)
query20 = """
SELECT 
    e.BusinessEntityID,
    DATEDIFF(DAY, MAX(eph.RateChangeDate), GETDATE()) AS DaysSinceLastPromotion
FROM 
    HumanResources.Employee AS e
JOIN 
    HumanResources.EmployeePayHistory AS eph
    ON e.BusinessEntityID = eph.BusinessEntityID
GROUP BY 
    e.BusinessEntityID;

"""
person_promotion = pd.read_sql(query20, engine)
person_promotion

,BusinessEntityID,DaysSinceLastPromotion
0,1,5725
1,2,6074
2,3,6155
3,4,4660
4,5,6099
...,...,...
285,286,4128
286,287,4537
287,288,4128
288,289,4493


In [58]:
#21. Analyze the effectiveness of employee development programs.						
query21 = """
SELECT 
    p.EmailPromotion, 
    COUNT(p.BusinessEntityID) AS TotalPeople
FROM 
    Person.Person AS p
GROUP BY 
    p.EmailPromotion;
"""
email_df = pd.read_sql(query21, engine)
print(email_df)


   EmailPromotion  TotalPeople
0               0        11158
1               1         5044
2               2         3770


In [40]:
# Query to find employees with the highest number of shifts
query22 = """
SELECT 
    e.BusinessEntityID as EmployeeID, 
    p.FirstName, 
    p.LastName, 
    COUNT(edh.ShiftID) as ShiftCount
FROM HumanResources.EmployeeDepartmentHistory edh
JOIN HumanResources.Employee e ON edh.BusinessEntityID = e.BusinessEntityID
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
GROUP BY e.BusinessEntityID, p.FirstName, p.LastName
ORDER BY ShiftCount DESC
"""
top_shift_employees_df = pd.read_sql(query22, engine)
print(top_shift_employees_df)


     EmployeeID FirstName           LastName  ShiftCount
0           250    Sheela               Word           3
1             4       Rob            Walters           2
2           224   William               Vong           2
3           234     Laura             Norman           2
4            16     David            Bradley           2
..          ...       ...                ...         ...
285         289       Jae                Pak           1
286         290    Ranjit  Varkey Chudukatil           1
287           1       Ken            Sánchez           1
288           2     Terri              Duffy           1
289           3   Roberto         Tamburello           1

[290 rows x 4 columns]


In [44]:
query23 = """
SELECT 
    e.BusinessEntityID as EmployeeID, 
    ABS(DATEDIFF(HOUR, s.EndTime, s.StartTime)) as ShiftDuration
FROM HumanResources.EmployeeDepartmentHistory edh
JOIN HumanResources.Employee e ON edh.BusinessEntityID = e.BusinessEntityID
JOIN HumanResources.Shift s ON edh.ShiftID = s.ShiftID
"""
compliance_df = pd.read_sql(query23, engine)
print(compliance_df)


     EmployeeID  ShiftDuration
0             1              8
1             2              8
2             3              8
3             4              8
4             4              8
..          ...            ...
291         213             16
292         218             16
293         226             16
294         229             16
295         231             16

[296 rows x 2 columns]


In [48]:
#24. Determine the distribution of employee performance ratings across departments.				
query24 = """
SELECT 
    d.Name AS DepartmentName,
    SUM(p.EmailPromotion) AS TotalPromotionEmails
FROM 
    Person.Person AS p
JOIN 
    HumanResources.Employee AS e ON p.BusinessEntityID = e.BusinessEntityID
JOIN 
    HumanResources.EmployeeDepartmentHistory AS edh ON e.BusinessEntityID = edh.BusinessEntityID
JOIN 
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
GROUP BY 
    d.Name
ORDER BY 
    TotalPromotionEmails DESC;


"""
shift_email_distribution_df = pd.read_sql(query24, engine)
shift_email_distribution_df['Rank'] = shift_email_distribution_df['TotalPromotionEmails'].rank(ascending=False)
print(shift_email_distribution_df.sort_values('Rank'))


                DepartmentName  TotalPromotionEmails  Rank
0                   Production                   129   1.0
1                        Sales                    10   2.5
2                    Marketing                    10   2.5
3                   Purchasing                     8   4.0
4         Information Services                     6   5.5
5                      Finance                     6   5.5
6                  Engineering                     5   7.0
7              Human Resources                     4  10.0
8            Quality Assurance                     4  10.0
9     Research and Development                     4  10.0
10      Shipping and Receiving                     4  10.0
11                 Tool Design                     4  10.0
12            Document Control                     2  13.5
13          Production Control                     2  13.5
14                   Executive                     1  15.0
15  Facilities and Maintenance                     0  16